In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'eryk-mazus/polka-1.1b'  # Updated model name to match initialization

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/pio/scratch/1/i317214/miniconda/envs/hallucination_detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = tokenizer.get_vocab()
# print(vocab)
vocab_switch_key = {v: k for k, v in vocab.items()}

def best_k(prefix, K=10):
    letter = prefix[0].lower()
    input_ids = tokenizer(prefix, return_tensors='pt')['input_ids'].to(device)    
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    d = {}
    # tokenizer.get_vocab()
    for i in range(probs.shape[0]):
        tok_str = tokenizer.decode(i, skip_special_tokens=True)
        tok_str = vocab_switch_key.get(i,"")
        # print(f"'{tok_str}'")
        # Remove leading special characters (e.g., 'Ġ' in some tokenizers)
        # tok_str_clean = tok_str.lstrip("Ġ")
        tok_str_clean = tok_str
        # Check if the token has an odd number of letters
        if (len(tok_str_clean) > 3 and tok_str_clean[0] != '▁') or \
            (len(tok_str_clean) > 1 and tok_str_clean[0] == '▁' and tok_str_clean[1] == letter):
            d[tok_str.replace('▁', ' ')] = probs[i].item()
            # print(f"asdasdasdsad---#{tok_str}#")
        # if i>1000:
        #     break
    
    # Sort tokens by probability in descending order and select top K
    sorted_tokens = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return sorted_tokens[:K]

def sample_from_pairs(pairs): 
    tokens  = [p[0] for p in pairs]    
    weights = [p[1] for p in pairs]
    if not tokens:
        return ""  # Return empty string if no valid tokens
    return random.choices(tokens, weights=weights, k=1)[0]

# Example prefixes
start_txts = \
"""Obowiązuje on od
Został zrodzony ze
Po pierwsze, projekt
Po Panthers przejechali
Duze dwusuwowe diesle
Niestety, nikt nie
Pani poseł, proszę
Proszę państwa, po
Proszę pana posła""".split('\n')

def sample_demo(N, txt):
    for i in range(N):
        d = best_k(txt)
        # print(f"PREFIX: '{txt}'")
        if not d:
            # print("   [INFO] No allowed tokens with odd number of letters found.")
            # print("="*60)
            break
        next_token = sample_from_pairs(d)
        for t, p in d:
            star = '*' if t == next_token else ''
            # print(f"   [{t}]{star} {p:.4f}")
        txt += next_token
        # print("="*60)
    print(f"COMPLETION: '{txt}'\n")

# Run the demo for each prefix
for start_txt in start_txts:
    print(f"Generating completion for prefix: '{start_txt}'")
    sample_demo(15, start_txt)  # Generate 3 tokens for each prefix
    # print("\n" + "#"*80 + "\n")


Generating completion for prefix: 'Obowiązuje on od'


COMPLETION: 'Obowiązuje on odroczone w odstępstwie od okresów wskazanych odprawione od og'

Generating completion for prefix: 'Został zrodzony ze'
COMPLETION: 'Został zrodzony ze złamanego źródła wiersza to źródło się, które zostało wybrane w źród'

Generating completion for prefix: 'Po pierwsze, projekt'
COMPLETION: 'Po pierwsze, projektowanie podnosi poczucie pewnego prestiżenia, co przyciąga przedsiębiorstwa'

Generating completion for prefix: 'Po Panthers przejechali'
COMPLETION: 'Po Panthers przejechali przez parkingu przy panieńskich pucharach <0xF0>\nPo panther prze'

Generating completion for prefix: 'Duze dwusuwowe diesle'
COMPLETION: 'Duze dwusuwowe diesle dostepne w dobrejach dla dealerem dla dealeria dacia duster'

Generating completion for prefix: 'Niestety, nikt nie'
COMPLETION: 'Niestety, nikt nie nastrudnił się na naszych nieszczeg...\nPrzepięknie utrzym'

Generating completion for prefix: 'Pani poseł, proszę'
COMPLETION: 'Pani poseł, proszę przestać się przepracować, p